In [74]:
#!/usr/bin/env python
# coding: utf-8

import sys
import os
import matplotlib.pyplot as plt
import glob

from datetime import datetime
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox
# import keras
import pandas as pd
import numpy as np
import random
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from functools import partial

class interactive_data_chooser:
    """
    Class for selecting data graphically and displaying it
    """
    def __init__(self, df, columns):
        # we don't need this dataframe, make a df_copy instead?
        self.outlier_df = pd.DataFrame()

        # self.df = df
        self.df_copy = df.copy()
        self.columns = columns
        self.df_copy["manual_outlier"] = -1
        self.df_copy["model_outlier"] = 0

        self.axis_dropdowns = None
        self.chosen_color_column = self.df_copy["manual_outlier"]
    
    def activate_plot(self):
        """
        Display interactive plot where images (data points in the plot)
        can be selected using box select or lasso select. 
        """
        # TODO: cmin and cmax depending on chosen_color_column (manual_outlier will always be -1 to 1) 
        # color_continuous_scale=[(0, "red"), (0.5, "green"), (1, "blue")]) maybe work to not have dynamic values?
        self.df_copy.reset_index(inplace=True,drop=True)
        numeric_df = self.df_copy.select_dtypes(include=np.number)
        numeric_columns = numeric_df.columns
        # y = self.df_copy[self.columns[0]], x = self.df_copy[self.columns[1]]
        # Hur sätter jag x- och y-axeln?
        # Kolla upp varför symbolerna inte ändras. Vad är det som visas? 
        # Varför lägger sig non-outlier-tracen som en matta innan jag ändrat axlarna?
        # Hur är det med self.f och trace? Något annat som är fel? 
        
        
        """ self.f = go.FigureWidget([go.Scatter(y = self.df_copy[self.columns[0]], 
                                             x = self.df_copy[self.columns[1]], 
                                             mode = 'markers',
                                             selected_marker_color = "red", 
                                             name="default",
                                             marker=dict(color=numeric_df[numeric_columns[0]],
                                                        colorbar=dict(thickness=10), 
                                                        colorscale=["blue", "green", "orange"]))]) """
        
        

        # Create the scatter plot with markers and lines for z < 1
        trace1 = go.Scatter(x=self.df_copy.loc[self.chosen_color_column < 1, 'x'], 
                                    y=self.df_copy.loc[self.chosen_color_column < 1, 'y1'],
                                    mode='markers+lines', 
                                    selected_marker_color = "red", 
                                    marker=dict(size=10, 
                                                colorscale=["blue", "green", "orange"], 
                                                color=numeric_df[numeric_columns[0]]),
                                                legendgroup="Trace",
                                                showlegend=True,
                                                name="non-outlier")
        """ self.f.add_trace(go.Scatter(x=self.df_copy.loc[self.chosen_color_column < 1, 'x'], 
                                    y=self.df_copy.loc[self.chosen_color_column < 1, 'y1'],
                                    mode='markers+lines', 
                                    selected_marker_color = "red", 
                                    marker=dict(size=10, 
                                                colorscale=["blue", "green", "orange"], 
                                                color=numeric_df[numeric_columns[0]]),
                                                legendgroup="Trace",
                                                showlegend=True,
                                                name="non-outlier")) """

        # Add a second scatter trace with markers only for z = 1
        trace2 = go.Scatter(x=self.df_copy.loc[self.chosen_color_column == 1]['x'], 
                                    y=self.df_copy.loc[self.chosen_color_column == 1]['y1'],
                                    mode='markers', 
                                    selected_marker_color = "red", 
                                    marker=dict(size=10, 
                                                colorscale=["blue", "green", "orange"], 
                                                color=numeric_df[numeric_columns[0]]),
                                                legendgroup="Trace", 
                                                showlegend=True,
                                                name="outlier")
        """ self.f.add_trace(go.Scatter(x=self.df_copy.loc[self.chosen_color_column == 1]['x'], 
                                    y=self.df_copy.loc[self.chosen_color_column == 1]['y1'],
                                    mode='markers', 
                                    selected_marker_color = "red", 
                                    marker=dict(size=10, 
                                                colorscale=["blue", "green", "orange"], 
                                                color=numeric_df[numeric_columns[0]]),
                                                legendgroup="Trace", 
                                                showlegend=True,
                                                name="outlier")) """
        
        
        
        trace1.hovertemplate = '<b>Trace 1</b><br>X: %{x}<br>Y: %{y}'
        trace2.hovertemplate = '<b>Trace 2</b><br>X: %{x}<br>Y: %{y}'
        
        self.f = go.FigureWidget(data=[trace1, trace2])

        # Customized legend
        self.f.add_trace(go.Scatter(y=[None], mode='markers',
                         marker=dict(symbol='circle', color='blue'),
                         name='Not manually chosen'
                         ))
        self.f.add_trace(go.Scatter(y=[None], mode='markers',
                         marker=dict(symbol='triangle-up', color='green'),
                         name='Not outlier',
                         ))
        self.f.add_trace(go.Scatter(y=[None], mode='markers',
                         marker=dict(symbol='x', color='orange'),
                         name='Outlier',
                         ))
        self.f.data[0].showlegend = True
        
        scatter = self.f.data[0]
        scatter.marker.opacity = 0.5
        
        self.axis_dropdowns = interactive(self.update_axes, yaxis = self.columns, xaxis = self.columns, color = numeric_columns)
        scatter.on_selection(self.selection_fn)
        
        # Put everything together
        return VBox((HBox(self.axis_dropdowns.children),self.f))
    
    def update_axes(self, xaxis, yaxis,color):
        scatter = self.f.data[0]
        scatter.x = self.df_copy[xaxis]
        scatter.y = self.df_copy[yaxis]
        scatter.marker.color = self.df_copy[color]
        with self.f.batch_update():
            self.f.layout.xaxis.title = xaxis
            self.f.layout.yaxis.title = yaxis
   
    def update_manual_outlier(self, row):
        row["manual_outlier"] = 1 if self.df_copy[row[0]]["manual_outlier"] != 1 else 0
        return row
    
    # def multiply_rows(row): Use this solution instead of iterrows
        # return row['column1'] * row['column2']

        # my_df['multiplied'] = my_df.apply(multiply_rows,axis=1)

    def update_temp_df_last_sel(self, row, last_selected):
        row["last_selected"] = last_selected
        return row
    
    """
    selected_indices = [p['pointIndex'] for p in points['points']]
    # set the skip mask to NaN for the selected points
    skip_mask[selected_indices] = np.nan
    # update the x and y data of the scatter trace with the skip mask applied
    trace.x = df['x'] * skip_mask
    trace.y = df['y'] * skip_mask

    """
    def skip_line_to_outlier(self, trace, points, selector):
        skip_mask = np.ones(self.df_copy.shape[0], dtype=bool)
        selected_indices = [p for p in points.point_inds]
        skip_mask[selected_indices] = False

        trace.x = self.df_copy[ "x"]
        trace.y = self.df_copy["y1"]
        trace.mode = "markers+lines"

        trace.update_traces(
            x=self.df_copy.loc[skip_mask, "x"],
            y=self.df_copy.loc[skip_mask, "y1"],
            mode='markers',
            marker=dict(symbol='x')

        )

        return trace
        

    def selection_fn(self,trace,points,selector):
        """
        Keeping track of points manually selected and change values in column ["manual_outlier"].
        Value for points not manually selected is -1. If selected to be an outlier, value is set to 1 (and displayed as an x).
        If selected again not to be an outlier, value is set to 0 (and displayed as a triangle). Previous value is stored for future 
        possibility to undo selection. TODO: Skip previous value since undo button is not required?

        Each selection is stored in a temp_df and all temp_df's are stored in self.outlier_df.
        The dataframe drop_duplicates_df is the df which will be used to train the model, where only 
        the last manually made change to a data point is included. 
        """
        
        temp_df = self.df_copy.loc[points.point_inds]
        # temp_df = trace något
        self.chosen_color_column = self.axis_dropdowns.children[2].value  
        
        last_selected = len(temp_df)
        
        # temp_df["last_selected"] = temp_df.apply(lambda row: self.update_temp_df_last_sel(row, last_selected), axis=1)
        for i in temp_df.iterrows():
            idx = i[0]
            temp_df.at[idx, "last_selected"] = last_selected
            # This is needed for keeping track of the changes
            temp_df.at[idx, "manual_outlier"] = 1 if self.df_copy.at[idx, "manual_outlier"] != 1 else 0
            # This is needed for displaying values in the plot
            self.df_copy.at[idx, "manual_outlier"] = 1 if self.df_copy.at[idx, "manual_outlier"] != 1 else 0

        self.outlier_df = pd.concat([self.outlier_df, temp_df], ignore_index=False, axis=0)

        # Needed? Just for printing in the notebook
        no_points = "point" if last_selected == 1 else "points"
        print(f"Selected {last_selected} new {no_points}. Total: {len(self.outlier_df)}")

        # Drop duplicates (needed?) 
        drop_duplicates_df = self.outlier_df.drop_duplicates(subset=["x", "y1"], keep="last")
        drop_duplicates_df.sort_values(by=["x"], inplace=True)
        print(f"Unique points selected ({len(drop_duplicates_df)}):")
        for i in drop_duplicates_df.iterrows():
            outlier = "yes" if i[1][3] == 1 else "no"
            print(f"x: {int(i[1][0])}, y1: {int(i[1][1])}, outlier: {outlier}")

        # Update color and symbols based on value in manual_outlier
        symbols = self.df_copy["manual_outlier"].map({-1: "circle", 0: "triangle-up", 1: "x"})
        trace.update(marker_color=self.df_copy[self.chosen_color_column], marker_symbol=symbols)
        
        # Switch trace of the selected data points to exclude outliers from connected data points
        """ points: Points(point_inds=[220, 221, 222],
                xs=[220, 221, 222],
                ys=[83, 90, 98],
                trace_name='non-outlier',
                trace_index=0)
              
            trace: Scatter({
                'hovertemplate': '<b>Trace 1</b><br>X: %{x}<br>Y: %{y}',
                'legendgroup': 'Trace',
                'marker': {'color': array([-1, -1, -1, ..., -1, -1, -1], dtype=int64),
                        'colorscale': [[0.0, 'blue'], [0.5, 'green'], [1.0, 'orange']],
                        'opacity': 0.5,
                        'size': 10,
                        'symbol': array(['circle', 'circle', 'circle', ..., 'circle', 'circle', 'circle'],
                                        dtype=object)},
                'mode': 'markers+lines',
                'name': 'non-outlier',
                'selected': {'marker': {'color': 'red'}},
                'selectedpoints': [293, 294, 296],
                'showlegend': True,
                'uid': '33a04aab-e909-4aac-8828-0eb63485f182',
                'x': array([  0,   1,   2, ..., 997, 998, 999]),
                'y': array([49, 71, 82, ...,  5, 64, 49], dtype=int64)
})  
            selected_trace_id: 0
            selected_point-idx: [287, 288]
              """
        selected_data = trace.selectedpoints
        """
        def update_traces(trace, points, selector):
            # get the data from the selected points in trace1
            selected_data = trace.data[0].selectedpoints
            x = [trace.data[0].x[i] for i in selected_data]
            y = [trace.data[0].y[i] for i in selected_data]

            # add the selected data to trace2
            trace2 = go.Scatter(x=x, y=y, mode='markers', marker=dict(color='blue'))
            f.add_trace(trace2)

            # remove the selected data from trace1
            trace.data[0].x = [trace.data[0].x[i] for i in range(len(trace.data[0].x)) if i not in selected_data]
            trace.data[0].y = [trace.data[0].y[i] for i in range(len(trace.data[0].y)) if i not in selected_data]

            # update the selected points in trace1
            trace.data[0].selectedpoints = []
        """

        # arr = np.array([[0,2,4],[6,8,10]]) 
        # app_arr = np.append(arr, [13,15,17])
        x = np.array([trace.x[i] for i in selected_data])
        y = np.array([trace.y[i] for i in selected_data])
        outlier_x = np.array(self.f.data[1].x)
        outlier_y = np.array(self.f.data[1].y)
        outlier_x = np.append(outlier_x, x)
        outlier_y = np.append(outlier_y, y)
        print(outlier_x)
        
        self.f.update_traces(y=outlier_y, x=outlier_x, selector=dict(name="outlier"))
        trace.x = [trace.x[i] for i in range(len(trace.x)) if i not in selected_data]
        trace.y = [trace.y[i] for i in range(len(trace.y)) if i not in selected_data]
        self.f.update_traces(y=trace.y, x=trace.x, selector=dict(name="non-outlier")) 
        print(f"trace: {trace}")
        print(f"self.f: {self.f}")
        # 1. Det som händer nu är att punkten jag markerat flyttas till trace2, alltså outlier, och blir en röd cirkel utanför linjen. Men nästa punkt i 
        # trace1 blir istället ett x trots att den inte markerats.
        # 2. Om jag sedan markerar punkten som felaktigt blivit ett x försvinner punkten jag markerade i förra omgången, punkten (alltså krysset) jag markerade nu
        # blir en röd cirkel och nästa punkt i trace1 blir en triangel.

        # Efter att ha lagt till outlier_x samt outlier_y flyttas inte längre punkterna över till trace2, utan får rätt symbol, men har fortfarande linjen kvar och stannar kvar.
        # get the trace ID of the selected points
        # selected_trace_id = points.trace_index  # 0, points.trace_name='non-outlier'
         
        # get the index of the selected points in the selected trace
        # selected_points_idx = points.point_inds
        
        
        # get the data of the selected points
        # selected_points = trace[selected_points_idx] ChatGPT
        # selected_points = {x, y, trace_name for x, y, name in trace}
        # print(f"selected_point: {selected_points}")
        """
        # remove the selected points from the selected trace
        trace[selected_points_idx] = None  # Do I need this?

        # add the selected points to the other trace
        other_trace_idx = 1 - selected_trace_id
        self.f.data[other_trace_idx].add_trace(selected_points) """

        # TODO: If it works, change this
        """ self.f.layout = go.Layout(
                title='Scatter Plot with Selections',
                xaxis=dict(title='X-Axis'),
                yaxis=dict(title='Y-Axis')
            ) """
        """
        
        # define a function to handle the on_selection event
        def handle_selection(trace, points, selector):
            # get the trace ID of the selected points
            selected_trace_id = points.trace_index[0]
            
            # get the index of the selected points in the selected trace
            selected_points_idx = points.point_inds
            
            # get the data of the selected points
            selected_points = trace[selected_points_idx]
            
            # remove the selected points from the selected trace
            trace[selected_points_idx] = None
            
            # add the selected points to the other trace
            other_trace_idx = 1 - selected_trace_id
            fig.data[other_trace_idx].add_trace(selected_points)
            
            # update the figure layout
            fig.layout = go.Layout(
                title='Scatter Plot with Selections',
                xaxis=dict(title='X-Axis'),
                yaxis=dict(title='Y-Axis')
            )

        # attach the handle_selection function to the on_selection event of the figure widget
        fig.data[0].on_selection(handle_selection)

        # set the layout of the figure
        fig.update_layout(title='Scatter Plot with Two Traces',
                        xaxis_title='X-Axis', yaxis_title='Y-Axis')

        # display the figure
        fig.show()

        """

        # This part is suppose to skip points that are marked as outliers, but it's not working
        """ skip_mask = np.ones(self.df_copy.shape[0], dtype=bool)
        selected_indices = [p for p in points.point_inds]
        skip_mask[selected_indices] = False

        trace.x = self.df_copy[ "x"]
        trace.y = self.df_copy["y1"]
        trace.mode = "markers+lines"

        trace.update_traces(
            x=self.df_copy.loc[skip_mask, "x"],
            y=self.df_copy.loc[skip_mask, "y1"],
            mode='markers',
            marker=dict(symbol='x') 
        ) """

    def clear_selection(self):
        self.outlier_df = self.outlier_df.iloc[0:0]
    
    def show_selected(self):
        for index, row in self.outlier_df.iterrows():
            plt.figure()
            plt.imshow(plt.imread(row['file']))
            plt.title(f"{row['time']}, wl: {row['wl']}, turb_s: {row['turb_sensor']}, turb_p: {row['turb_post']}")

    # create train model function based on outlier status in self.df

    # visualize result in graph

    # function to mark point as non-outlier DONE

    # button to undo choice

    # button to confirm (then train model), disable if not choosen areas == 1



In [75]:
#!/usr/bin/env python
# coding: utf-8

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

def create_fake_df(n):
    """
    Creates a dataframe with n rows and columns "x", "y1" and "y2". 
    The data are integers, 0-100.
    """
    x = []
    y1 = []
    y2 = []

    for i in range(n):    
        x_int = i
        x.append(x_int)
        y1_int = random.randint(0, 100)
        y1.append(y1_int)
        y2_int = random.randint(0, 100)
        y2.append(y2_int)

    int_dict = {"x": np.sort(x), "y1": y1, "y2": y2}
    df = pd.DataFrame(int_dict)
    return df

df = create_fake_df(1000)
chooser = interactive_data_chooser(df, df.columns)
chooser.activate_plot()


Selected 2 new points. Total: 2
Unique points selected (2):
x: 336, y1: 61, outlier: yes
x: 338, y1: 67, outlier: yes
[336 338]
trace: Scatter({
    'hovertemplate': '<b>Trace 1</b><br>X: %{x}<br>Y: %{y}',
    'legendgroup': 'Trace',
    'marker': {'color': array([-1, -1, -1, ..., -1, -1, -1], dtype=int64),
               'colorscale': [[0.0, 'blue'], [0.5, 'green'], [1.0, 'orange']],
               'opacity': 0.5,
               'size': 10,
               'symbol': array(['circle', 'circle', 'circle', ..., 'circle', 'circle', 'circle'],
                               dtype=object)},
    'mode': 'markers+lines',
    'name': 'non-outlier',
    'selected': {'marker': {'color': 'red'}},
    'selectedpoints': [336, 338],
    'showlegend': True,
    'uid': '78b732c9-bb80-4a51-bcae-6f1ee1c6674d',
    'x': [0, 1, 2, ..., 997, 998, 999],
    'y': [35, 68, 88, ..., 60, 40, 51]
})
self.f: FigureWidget({
    'data': [{'hovertemplate': '<b>Trace 1</b><br>X: %{x}<br>Y: %{y}',
              'legendg

C:\Users\johanna4303\AppData\Local\Temp\ipykernel_16348\3136511465.py:228: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Selected 2 new points. Total: 7
Unique points selected (4):
x: 334, y1: 44, outlier: no
x: 335, y1: 23, outlier: no
x: 336, y1: 61, outlier: no
x: 338, y1: 67, outlier: yes
[336 338 334 335 340 337 339]
trace: Scatter({
    'hovertemplate': '<b>Trace 1</b><br>X: %{x}<br>Y: %{y}',
    'legendgroup': 'Trace',
    'marker': {'color': array([-1, -1, -1, ..., -1, -1, -1], dtype=int64),
               'colorscale': [[0.0, 'blue'], [0.5, 'green'], [1.0, 'orange']],
               'opacity': 0.5,
               'size': 10,
               'symbol': array(['circle', 'circle', 'circle', ..., 'circle', 'circle', 'circle'],
                               dtype=object)},
    'mode': 'markers+lines',
    'name': 'non-outlier',
    'selected': {'marker': {'color': 'red'}},
    'selectedpoints': [334, 335],
    'showlegend': True,
    'uid': '78b732c9-bb80-4a51-bcae-6f1ee1c6674d',
    'x': [0, 1, 2, ..., 997, 998, 999],
    'y': [35, 68, 88, ..., 60, 40, 51]
})
self.f: FigureWidget({
    'data': [{'hov

C:\Users\johanna4303\AppData\Local\Temp\ipykernel_16348\3136511465.py:228: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
import plotly.graph_objects as go
import pandas as pd

# Create a sample dataframe
df = pd.DataFrame({'x': [0.5, 1.5, 2.5, 3.5], 'y': [1, 2, 3, 2], 'z': [0.2, 0.5, 1, 0.7]})

# Create the scatter plot with markers and lines for z < 1
fig = go.Figure()
fig.add_trace(go.Scatter(x=df[df['z'] < 1]['x'], y=df[df['z'] < 1]['y'],
                         mode='markers+lines', line=dict(color='blue'), marker=dict(size=10, color=df[df['z'] < 1]['z']),
                         name='z < 1'))

# Add a second scatter trace with markers only for z = 1
fig.add_trace(go.Scatter(x=df[df['z'] == 1]['x'], y=df[df['z'] == 1]['y'],
                         mode='markers', marker=dict(size=10, color='red'),
                         name='z = 1'))

# Update the color axis range to include values below 1
fig.update_layout(coloraxis_colorbar=dict(tickvals=[0, 0.5, 1]))

# Show the plot
fig.show()





In [40]:
import plotly.graph_objs as go
import numpy as np

# generate some sample data for two traces
x1 = np.random.normal(0, 1, 100)
y1 = np.random.normal(0, 1, 100)
x2 = np.random.normal(2, 1, 100)
y2 = np.random.normal(2, 1, 100)

# create two scatter trace objects with different colors and names
trace1 = go.Scatter(x=x1, y=y1, mode='markers', name='Trace 1', marker=dict(color='blue'))
trace2 = go.Scatter(x=x2, y=y2, mode='markers', name='Trace 2', marker=dict(color='red'))

# create a figure with the two scatter trace objects
fig = go.FigureWidget(data=[trace1, trace2])
trace = None
selected_points = None
# define a function to handle the on_selection event
def handle_selection(trace, points, selector):
    # get the trace ID of the selected points
    selected_trace_id = points.trace_index[0]
    
    # get the index of the selected points in the selected trace
    selected_points_idx = points.point_inds
    
    # get the data of the selected points
    selected_points = trace[selected_points_idx]
    
    # remove the selected points from the selected trace
    trace[selected_points_idx] = None
    
    # add the selected points to the other trace
    other_trace_idx = 1 - selected_trace_id
    fig.data[other_trace_idx].add_trace(selected_points)
    
    # update the figure layout
    fig.layout = go.Layout(
        title='Scatter Plot with Selections',
        xaxis=dict(title='X-Axis'),
        yaxis=dict(title='Y-Axis')
    )

# attach the handle_selection function to the on_selection event of the figure widget
fig.data[0].on_selection(handle_selection)

# set the layout of the figure
fig.update_layout(title='Scatter Plot with Two Traces',
                  xaxis_title='X-Axis', yaxis_title='Y-Axis')


# display the figure
fig.show()









In [3]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create example data
import numpy as np
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

# Create scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))

# Add dropdown menu
updatemenus = [{'buttons': [{'args': [{'selectedpoints': [0]}],
                              'label': 'Mean',
                              'method': 'restyle'},
                             {'args': [{'selectedpoints': [0]}],
                              'label': 'Median',
                              'method': 'restyle'}],
                'direction': 'down',
                'showactive': True}]
fig.update_layout(updatemenus=updatemenus)

# Add box select event handler
fig.add_layout_image(dict(
    source="https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/gitbook/images/lasso.png",
    xref="paper", yref="paper",
    x=1.05, y=1.2,
    sizex=0.2, sizey=0.2,
    xanchor="right", yanchor="bottom"
))

fig.update_layout(
    dragmode="select",
    annotations=[
        dict(
            text="Draw a box around the points to select",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.5, y=1.1, font_size=16
        )
    ]
)

fig.update_layout(
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title",
    title="Box Select and Dropdown Example"
)

# Define callback function for dropdown menu
def update_fig(selection):
    if selection == 'Mean':
        fig.update_traces(marker=dict(color='red', size=10))
        selected_points = fig.data[0].selectedpoints
        if selected_points:
            mean_value = np.mean(y[selected_points])
            fig.add_annotation(text=f"Mean: {mean_value:.2f}", xref="paper", yref="paper",
            x=0.8, y=0.9, showarrow=False, font_size=16)
    elif selection == 'Median':
        fig.update_traces(marker=dict(color='green', size=10))
        selected_points = fig.data[0].selectedpoints
        if selected_points:
            median_value = np.median(y[selected_points])
            fig.add_annotation(text=f"Median: {median_value:.2f}", xref="paper", yref="paper",
            x=0.8, y=0.8, showarrow=False, font_size=16)

# Set up callback for dropdown menu
fig.for_each_trace(lambda trace: trace.on_selection(lambda x,y,trace=trace: update_fig('Mean')))

fig.show()


In [2]:
import plotly.graph_objs as go
from ipywidgets import Button, Dropdown
from IPython.display import display

# Create example data
x = [1, 2, 3, 4, 5]
y = [10, 20, 30, 40, 50]

# Create plotly plot with box select enabled
trace = go.Scatter(x=x, y=y, mode='markers')
layout = go.Layout(title='Selected Points', xaxis=dict(range=[0, 6]), yaxis=dict(range=[0, 60]), dragmode='select')
fig = go.Figure(data=[trace], layout=layout)

# Define event handler function to display dropdown menu
def handle_selection(event):
    if event is None or len(event) == 0:
        return
    
    # Create the dropdown menu with the selected values
    dropdown_options = [
        {'label': 'Mean', 'value': 'mean'},
        {'label': 'Max', 'value': 'max'},
        {'label': 'Min', 'value': 'min'},
    ]
    dropdown = Dropdown(options=dropdown_options, description='Select an operation')
    display(dropdown)
    
# Add event handler function to the plotly plot
fig.layout.on_change(handle_selection, 'selectedpoints')

# Create a button to clear the selection and hide the dropdown menu
button = Button(description='Clear Selection')
button.on_click(lambda _: dropdown.close() if 'dropdown' in globals() else None)
display(fig, button)




ValueError: Invalid property specification(s): ['selectedpoints']

In [5]:
glen_1 = pd.read_csv("data/asset-data-export_O12QnL6kAl-640876dfe5066-1678276319.csv")
glen_1

,"Asset name;""NU4202 Naset"""
"Exported on;""8 March 2023","12:51:59"""
"Timezone;""UTC +1""",NaN
"Date/Time;""Water level","Nap (cm)"""
2022-03-08 00:00:04;24,NaN
2022-03-08 00:30:00;24,NaN
...,...
2023-03-07 23:00:00;26,NaN
2023-03-07 23:30:00;25,NaN
2023-03-08 00:00:04;25,NaN
2023-03-08 00:30:00;25,NaN


In [6]:
glen_2 = pd.read_csv("data/asset-data-export_Zga3AM63oO-64098cbc36e23-1678347452.csv")
glen_2

,Asset name;NU4201 Spektrumgatan
Exported on;9 March 2023,08:37:32
Timezone;UTC +1,NaN
;,NaN
Date/Time;Water level,Nap (cm)
2022-03-09 00:00;5,NaN
...,...
2023-03-08 23:00;1,NaN
2023-03-08 23:30;1,NaN
2023-03-09 00:00;1,NaN
2023-03-09 00:30;1,NaN


In [12]:
glen_3 = pd.read_csv("data/asset-data-export_vEm3Jd5916-64098d190a5ca-1678347545.csv")
glen_3.head()

,Asset name;NU3328 Strandvägen
Exported on;9 March 2023,08:39:05
Timezone;UTC +1,NaN
;,NaN
Date/Time;Water level,Nap (cm)
2022-04-22 12:47;11,NaN
